In [110]:
import numpy as np
np.random.seed(42)

In [111]:
class Node:
    node_count = 0
    all_nodes = []
    def __init__(self, age=0.0):
        self.id = Node.node_count
        Node.node_count += 1
        Node.all_nodes.append(self)

        self.age = age

        self.parent = None
        self.children = []
        self.mutations = set()
    
    def print(self, indent=0):
        print("--" * indent + str(self.id) + " (" + str(self.age) + ")" + " n_mutations: " + str(len(self.mutations)))
        for child in self.children:
            child.print(indent+1)
    
    def get_leaves(self):
        if len(self.children) == 0:
            return [self]
        return [leaf for child in self.children for leaf in child.get_leaves()]
    
    def get_root(self):
        if self.parent is None:
            return self
        return self.parent.get_root()
    
    def adopt(self, child):
        assert child.parent is None
        assert child not in self.children
        self.children.append(child)
        child.parent = self

In [112]:
def coalesce(pop_size, sample_size, max_t=None):
    age = 0.0

    orphans = [Node() for _ in range(sample_size)]

    while len(orphans) > 1:
        tmean = 2 * pop_size / (sample_size * (sample_size - 1))
        age += np.random.exponential(tmean)

        if max_t is not None and age > max_t:
            break

        # Pick two orphans to coalesce, remove then from the list of orphas
        node1 = np.random.choice(orphans)
        orphans.remove(node1)

        node2 = np.random.choice(orphans)
        orphans.remove(node2)

        # Create a new node that is the parent of the two orphans
        parent = Node(age=age)
        parent.adopt(node1)
        parent.adopt(node2)

        # Add the parent to the list of orphans
        orphans.append(parent)

    return orphans

In [113]:
nodes = coalesce(10000, 10)

for node in nodes:
    node.print()
    print()

18 (833.6043876016665) n_mutations: 0
--17 (623.2997489572668) n_mutations: 0
----13 (359.5141961500335) n_mutations: 0
------3 (0.0) n_mutations: 0
------12 (155.27371756619692) n_mutations: 0
--------11 (141.97621343093454) n_mutations: 0
----------2 (0.0) n_mutations: 0
----------9 (0.0) n_mutations: 0
--------6 (0.0) n_mutations: 0
----15 (417.0129053481971) n_mutations: 0
------14 (372.41757451493856) n_mutations: 0
--------8 (0.0) n_mutations: 0
--------1 (0.0) n_mutations: 0
------0 (0.0) n_mutations: 0
--16 (497.6248452283031) n_mutations: 0
----10 (104.28179777263536) n_mutations: 0
------7 (0.0) n_mutations: 0
------4 (0.0) n_mutations: 0
----5 (0.0) n_mutations: 0



In [114]:
class Mutation:
    mut_count = 0
    @staticmethod
    def new():
        Mutation.mut_count += 1
        return Mutation.mut_count


In [115]:
def drop_mutations(node, mut_rate, time):
    mean_mutations = mut_rate * (time - node.age)
    num_mutations = np.random.poisson(mean_mutations)
    node.mutations = set(Mutation.new() for _ in range(num_mutations))
    for child in node.children:
        drop_mutations(child, mut_rate, node.age)

In [116]:
class Individual:
    def __init__(self, pop_size, sample_size, mut_rate, age = 0.0):
        self.pop_size = pop_size
        self.age = age

        self.mut_rate = mut_rate

        self.root = Node(age=age)
        self.sample = [Node() for _ in range(sample_size)]
        
        for node in self.sample:
            self.root.adopt(node)
    
    def print(self):
        self.root.print()

    def evolve(self, t):
        self.age += t

        orphans = coalesce(self.pop_size, len(self.sample), max_t=t)

        # assert all orphans have no parent
        assert all([orphan.parent is None for orphan in orphans])

        # pick parents for the orphans
        parents = np.random.choice(self.sample, size=len(orphans), replace=False)

        # adopt the orphans
        for parent, orphan in zip(parents, orphans):
            parent.adopt(orphan)

        for orphan in orphans:
            drop_mutations(orphan, self.mut_rate, t)

In [117]:
# Test the Node class
root = Node(age=0.0)
child1 = Node(age=1.0)
child2 = Node(age=2.0)

root.adopt(child1)
root.adopt(child2)

print("Tree structure:")
root.print()

print("\nLeaves of the tree:")
leaves = root.get_leaves()
for leaf in leaves:
    print(f"Leaf ID: {leaf.id}, Age: {leaf.age}")

print("\nRoot of the tree:")
root_node = child1.get_root()
print(f"Root ID: {root_node.id}, Age: {root_node.age}")

# Test the coalesce function
print("\nCoalescence process:")
nodes = coalesce(10000, 5)
for node in nodes:
    node.print()
    print()

Tree structure:
19 (0.0) n_mutations: 0
--20 (1.0) n_mutations: 0
--21 (2.0) n_mutations: 0

Leaves of the tree:
Leaf ID: 20, Age: 1.0
Leaf ID: 21, Age: 2.0

Root of the tree:
Root ID: 19, Age: 0.0

Coalescence process:
30 (2173.6104095303886) n_mutations: 0
--22 (0.0) n_mutations: 0
--29 (1276.105688220628) n_mutations: 0
----28 (312.7600113137961) n_mutations: 0
------23 (0.0) n_mutations: 0
------27 (47.789611263771754) n_mutations: 0
--------25 (0.0) n_mutations: 0
--------26 (0.0) n_mutations: 0
----24 (0.0) n_mutations: 0



In [118]:
# Test the Individual class
print("\nIndividual evolution:")
individual = Individual(pop_size=10000, sample_size=7, mut_rate=0.1)
individual.print()
print("\nIndividual after 10000s:")
individual.evolve(10000)
individual.print()


Individual evolution:
31 (0.0) n_mutations: 0
--32 (0.0) n_mutations: 0
--33 (0.0) n_mutations: 0
--34 (0.0) n_mutations: 0
--35 (0.0) n_mutations: 0
--36 (0.0) n_mutations: 0
--37 (0.0) n_mutations: 0
--38 (0.0) n_mutations: 0

Individual after 10000s:
31 (0.0) n_mutations: 0
--32 (0.0) n_mutations: 0
--33 (0.0) n_mutations: 0
--34 (0.0) n_mutations: 0
--35 (0.0) n_mutations: 0
--36 (0.0) n_mutations: 0
----51 (3362.581028846562) n_mutations: 684
------46 (936.0415622732556) n_mutations: 228
--------45 (0.0) n_mutations: 92
--------43 (0.0) n_mutations: 111
------50 (3300.603774582539) n_mutations: 11
--------49 (2751.6748397736524) n_mutations: 46
----------39 (0.0) n_mutations: 271
----------47 (1221.1582974382422) n_mutations: 145
------------40 (0.0) n_mutations: 111
------------44 (0.0) n_mutations: 132
--------48 (2578.6806207409104) n_mutations: 87
----------41 (0.0) n_mutations: 285
----------42 (0.0) n_mutations: 255
--37 (0.0) n_mutations: 0
--38 (0.0) n_mutations: 0
